**"사기" 단어 검색 후 101개 판례 데이터 크롤링**  
전처리: (숫자) [숫자] / , ' "  
날짜: ~20100101 이후 사건

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/SKT_FLY_AI/Project/AI_Model/Similarity/law_no.csv', encoding='euc-kr')
data.head(10)

,fact_one,caseNm,courtNm,judmnAdjuDe,caseNo
0,[1] 수 죄의 기본적 사실관계가 동일한지 여부의 판단 기준[2] 과실로 교통사고를...,살인미수·사기·사기미수,대법원,2010.2.25.,2009도14263
1,[1] 재정신청서 기재요건을 위반한 재정신청을 인용한 공소제기결정의 잘못을 그 본안...,사기,대법원,2010.11.11.,2009도224
2,"[1] 구 조세범 처벌법 에서 정한 자가 조세를 포탈한 경우, 포탈세액의 산정 기준...",특정범죄가중처벌등에관한법률위반(알선수재)·특정범죄가중처벌등에관한법률위반(조세)·증권...,대법원,2011.6.30.,2010도10968
3,[1] 구 조세범 처벌법 에서 정한 ‘사기 기타 부정한 행위’의 의의 및 판단 기준...,특정범죄가중처벌등에관한법률위반(조세)·조세범처벌법위반,대법원,2011.3.24.,2010도13345
4,[1] 통역인이 사건에 관하여 증인으로 증언한 경우 통역인 제척사유에 해당하는지 여...,특정경제범죄가중처벌등에관한법률위반(사기)·사문서위조·위조사문서행사·사기,대법원,2011.4.14.,2010도13583
5,"[1] 장물죄에서 본범이 되는 범죄행위에 대하여 우리 형법이 적용되지 않는 경우, ...",사기·장물취득·외국환거래법위반,대법원,2011.4.28.,2010도15350
6,[1] 피고인이 보험금을 편취할 의사로 허위 또는 과장하여 보험사고를 신고하거나 고...,사기,대법원,2011.2.24.,2010도17512
7,[1] 법원이 공소장 변경 없이 직권으로 공소장에 기재된 공소사실과 다른 범죄사실을...,사기,대법원,2010.4.29.,2010도2414
8,"[1] 항소법원이 피고인에게 소송기록 접수통지를 2회에 걸쳐 한 경우, 항소이유서 ...",사기,대법원,2010.5.27.,2010도3377
9,"[1] 사기 범행의 피해자로부터 현금을 예금계좌로 송금받은 경우, 그 사기죄의 객체...",사기방조·장물취득·전자금융거래법위반,대법원,2010.12.9.,2010도6256


In [ ]:
# fact_one 피처만 추출

fact = data['fact_one']
print(fact.head(10))

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
import numpy as np

# cp949 인코딩을 사용하여 CSV 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/SKT_FLY_AI/Project/AI_Model/Similarity/law.csv', encoding='cp949')

# 사용자 입력 텍스트 예시
user_input_text = "부동산 전세를 어제 계약했는데 사기 당한 거 같아. 보증금 빨리 돌려 받고 싶어."

# CountVectorizer 초기화 및 학습
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(df['fact_one'].append(pd.Series(user_input_text)))

# 사용자 입력 벡터 이진화
user_input_vector = np.where(count_matrix[-1].toarray() > 0, 1, 0)

# 자카드 유사도 계산
jaccard_sim = []
for case_vector in count_matrix[:-1]:
    case_vector_binary = np.where(case_vector.toarray() > 0, 1, 0)
    sim_score = jaccard_score(user_input_vector, case_vector_binary, average='micro')
    jaccard_sim.append(sim_score)

# 가장 유사한 판례의 인덱스 찾기
most_similar_case_index = jaccard_sim.index(max(jaccard_sim))
most_similar_jaccard_similarity = jaccard_sim[most_similar_case_index]

# 가장 유사한 판례의 caseNo 출력
most_similar_case_no = df.iloc[most_similar_case_index]['caseNo']
print(most_similar_case_no, most_similar_jaccard_similarity)

<ipython-input-6-6751ffcc02e3>:14: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  count_matrix = count_vectorizer.fit_transform(df['fact_one'].append(pd.Series(user_input_text)))


2019다301623 0.02857142857142857


In [8]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 31.2 MB/s eta 0:00:00


In [12]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt

# 한국어 텍스트 정제 함수
def clean_text(text):
    cleaned_text = re.sub('[^가-힣\s]', '', text)  # 한글과 공백을 제외한 모든 문자 제거
    return cleaned_text

# 불용어 제거 함수
def remove_stopwords(text, stopwords):
    words = text.split()
    result = [word for word in words if word not in stopwords]
    return ' '.join(result)

# 전처리 함수
def preprocess(text, stopwords):
    cleaned_text = clean_text(text)
    okt = Okt()
    tokens = okt.morphs(cleaned_text, stem=True)  # 형태소 분석기를 사용하여 토큰화 및 어간 추출
    filtered_text = remove_stopwords(' '.join(tokens), stopwords)
    return set(filtered_text.split())  # 집합으로 변환하여 반환

# 불용어 리스트
stopwords = ["는", "은", "이", "가", "을", "를", "등", "와", "과"]

# CSV 파일 불러오기 (파일 경로는 상황에 맞게 조정하세요)
df = pd.read_csv('/content/drive/MyDrive/SKT_FLY_AI/Project/AI_Model/Similarity/law_no.csv', encoding='cp949')

# 판례 데이터에서 'fact_one' 열만 추출하여 전처리
preprocessed_data_sets = [preprocess(text, stopwords) for text in df['fact_one']]

# 사용자 입력 텍스트 예시
user_input_text = "부동산 전세를 어제 계약했는데 사기 당한 거 같아. 보증금 빨리 돌려 받고 싶어."
preprocessed_user_input_set = preprocess(user_input_text, stopwords)

# Jaccard 유사도 계산 함수
def compute_jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

# 각 문서에 대해 Jaccard 유사도 계산
jaccard_similarities = [compute_jaccard_similarity(preprocessed_user_input_set, data_set) for data_set in preprocessed_data_sets]

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(jaccard_similarities)[-3:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = df.iloc[idx]['caseNo']
    similarity = jaccard_similarities[idx]
    fact_one = df.iloc[idx]['fact_one']
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Fact One: {fact_one}\n')

CaseNo: 2015도6905, Similarity: 0.05357142857142857
Fact One: 보험계약자가 보험계약 체결 시 보험금액이 목적물의 가액을 현저하게 초과하는 초과보험 상태를 의도적으로 유발한 후 보험사고가 발생하자 초과보험 사실을 알지 못하는 보험자에게 목적물의 가액을 묵비한 채 보험금을 청구하여 교부받은 경우, 보험금을 청구한 행위가 사기죄의 실행행위로서 기망행위에 해당하는지 여부(한정 적극)

CaseNo: 2020도12279, Similarity: 0.05128205128205128
Fact One: 소송촉진 등에 관한 특례법  에 따른 배상명령제도의 취지 / 같은 법  에서 정한 ‘다른 절차에 따른 손해배상청구’의 의미 및 이에 해당하는 경우 법원이 취할 조치

CaseNo: 2021도7831, Similarity: 0.05
Fact One: 기망행위에 의하여 국가적 또는 공공적 법익을 침해한 경우, 형법상 사기죄가 성립하기 위한 요건 / 기망행위에 의하여 조세를 포탈하거나 조세의 환급·공제를 받은 경우, 형법상 사기죄가 성립하는지 여부(소극)



In [14]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt

# 한국어 텍스트 정제 함수
def clean_text(text):
    cleaned_text = re.sub('[^가-힣\s]', '', text)  # 한글과 공백을 제외한 모든 문자 제거
    return cleaned_text

# 불용어 제거 함수
def remove_stopwords(text, stopwords):
    words = text.split()
    result = [word for word in words if word not in stopwords]
    return ' '.join(result)

# 전처리 함수
def preprocess(text, stopwords):
    cleaned_text = clean_text(text)
    okt = Okt()
    tokens = okt.morphs(cleaned_text, stem=True)  # 형태소 분석기를 사용하여 토큰화 및 어간 추출
    filtered_text = remove_stopwords(' '.join(tokens), stopwords)
    return set(filtered_text.split())  # 집합으로 변환하여 반환

# 불용어 리스트
stopwords = ["는", "은", "이", "가", "을", "를", "등", "와", "과"]

# CSV 파일 불러오기 (파일 경로는 상황에 맞게 조정하세요)
df = pd.read_csv('/content/drive/MyDrive/SKT_FLY_AI/Project/AI_Model/Similarity/law_add.csv', encoding='cp949')

# 판례 데이터에서 'fact_one' 열만 추출하여 전처리
preprocessed_data_sets = [preprocess(text, stopwords) for text in df['fact_one']]

# 사용자 입력 텍스트 예시
user_input_text = "부동산을 통해서 어제 전세 계약을 했는데 사기를 당함. 보증금을 돌려받고자 했으나 인정이 되지 않는 상황"
preprocessed_user_input_set = preprocess(user_input_text, stopwords)

# Jaccard 유사도 계산 함수
def compute_jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

# 각 문서에 대해 Jaccard 유사도 계산
jaccard_similarities = [compute_jaccard_similarity(preprocessed_user_input_set, data_set) for data_set in preprocessed_data_sets]

# 상위 3개 유사한 사건의 인덱스 찾기
top_indices = np.argsort(jaccard_similarities)[-3:][::-1]

# 상위 3개 유사한 사건 출력
for idx in top_indices:
    case_no = df.iloc[idx]['caseNo']
    similarity = jaccard_similarities[idx]
    fact_one = df.iloc[idx]['fact_one']
    print(f'CaseNo: {case_no}, Similarity: {similarity}')
    print(f'Fact One: {fact_one}\n')

CaseNo: 2009다7886378870, Similarity: 0.10606060606060606
Fact One: 부동산 중개업자가 중개를 하지 않았음에도 거래계약서 등을 작성·교부하는 것이 허용되는지 여부부동산 중개업자가 자신의 중개로 전세계약이 체결되지 않았음에도 실제 계약당사자가 아닌 자에게 전세계약서 등을 작성·교부해 줌으로써 이를 담보로 제공받아 금전을 대여한 대부업자가 대여금을 회수하지 못하는 손해를 입은 사안에서 중개업자의 주의의무 위반에 따른 손해배상책임을 인정한 사례

CaseNo: 2022도7940, Similarity: 0.08333333333333333
Fact One: 항소심에서 피고인이 불출석한 상태에서 그 진술 없이 판결하기 위해서는 피고인이 적법한 공판기일 통지를 받고서도 2회 연속으로 정당한 이유 없이 출정하지 않은 경우에 해당하여야 하는지 여부  이때 ‘적법한 공판기일 통지’에 공판기일 변경명령을 송달받은 경우도 포함되는지 여부

CaseNo: 2019다301623, Similarity: 0.07142857142857142
Fact One: 구 국세기본법 의2  에서 정한 ‘사기 기타 부정한 행위’의 의미 및 단순히 세법상 신고를 하지 않거나 허위의 신고를 한 행위가 이에 해당하는지 여부명의를 위장하여 소득을 얻는 행위가 구 국세기본법 의2  에서 정한 ‘사기 기타 부정한 행위’에 해당하는 경우

